In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

## Setup Bibliotecas

Para instalar las bibliotecas que necesitamos en esta práctica, quiten el símbolo de comenatrio y ejecuten esta celda. 

Sólo es necesario hacerlo una vez, por eso están comentadas por default.

In [ ]:
#! pip install --upgrade google-cloud-storage
#! pip install --upgrade google-cloud-bigquery-storage
#! pip install --upgrade google-cloud-bigquery[bqstorage,pandas]

# BigQuery

## Intro

El objetivo de esta clase es presentarles una introducción básica a BigQuery, para que sean capaces de descargar datasets que les puedan ser útiles y que conozcan cómo usar BigQuery en la creación de modelos de machine learning.


## Imports

In [ ]:
import os
from google.cloud import bigquery
#from google.cloud import bigquery_storage_v1beta1
import google.auth
import pandas as pd

## Creación de una cuenta

* Crear un usuario en la consola de Google Cloud https://console.cloud.google.com/

Con una cuenta de google (**no es necesario poner datos de tarjeta de crédito ni activar una cuenta con $300 dolares iniciales bonificados**)

* Crear un proyecto: clase-37

* Configurar la autenticación:

    En Cloud Console, en la página Crear una clave de cuenta de servicio.
https://console.cloud.google.com/apis/credentials/serviceaccountkey

    Seleccionamos el proyecto clase-37
    
    Seleccionar Crear Cuenta de servicio

    En paso 1, Ingresar un nombre en el campo Nombre de cuenta de servicio, y click en Crear y Continuar.

    En paso 2, Seleccionamos la función Propietario. Y click en boton LISTO.

    Aparece una lista con la cuenta de servicio. Seleccionar en Acciones, Administrar claves.
    
    Click en Agregar claves, y Crear clave nueva.
    
    Dejar seleccionado JSON y click en Crear. Se descargará un archivo JSON que contiene tus claves a tu computadora.

## Setup local de Google Account

Paso 3 de https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries?authuser=1

Seteo como variable de entorno GOOGLE_APPLICATION_CREDENTIALS el path del json que descargué en el paso anterior:

In [ ]:
#credential_path = "C:\\Users\\ncopo\\Desktop\\clase-53-ea6d944e9e65.json"
credential_path = "/media/paulati/Nuevo vol/paula/dh/2021/dsad_2021_paula/M7/CLASE_53/Data/clase-37-e9f8bf18d574.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

## Consola BigQuery

En esta parte vamos a ejecutar algunas consultas en la consola de BigQuery sobre datos disponibles en bigquery-public-data

https://console.cloud.google.com/bigquery?project=bigquery-public-data


https://console.cloud.google.com/bigquery

Ejecutemos esta consulta y guardemos los resultados en un archivo cvs en nuestra máquina.
<code>
SELECT corpus
FROM `bigquery-public-data.samples.shakespeare`
GROUP BY corpus;
</code>


Otras consultas 

https://www.kaggle.com/paultimothymooney/how-to-query-the-bigquery-sample-tables

Podemos explorar los datos disponibles usando la caja de búsqueda de Recursos (en el panel izquierdo)

https://console.cloud.google.com/bigquery?project=bigquery-public-data&authuser=2&p=bigquery-public-data&d=covid19_ecdc&page=dataset


## Query

https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries?authuser=2

**Documentación**

Client: 
https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html

query:
https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client.query

QueryJob: 
https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob

QueryJob.result: 
https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.result


RowIterator: 
https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.table.RowIterator.html#google.cloud.bigquery.table.RowIterator

In [ ]:
client = bigquery.Client()

In [ ]:
query_job = client.query("""
    SELECT
      CONCAT(
        'https://stackoverflow.com/questions/',
        CAST(id as STRING)) as url,
      view_count
    FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE tags like '%google-bigquery%'
    ORDER BY view_count DESC
    LIMIT 10""")

results = query_job.result()  # Waits for job to complete.


Veamos como iterar sobre los resultados

In [ ]:
results.total_rows
#results.to_dataframe()
itr = iter(results)
first_row = next(itr)

In [ ]:
print(results.total_rows)

In [ ]:
first_row

In [ ]:
second_row = next(itr)
second_row

Para volver a iterar sobre los resultados de la consulta, necesitamos repetirla.

Observar que si lo corro dos veces da error porque el iterador llego al final


In [ ]:
for row in results:
    print("{} : {} views".format(row.url, row.view_count))

Entonces repetimos la consulta usando el mismo objeto `QueryJob` para poder volver a recorrer los resultados:

In [ ]:
results = query_job.result()  # Waits for job to complete.

Podemos crear un objeto de tipo `pandas.DataFrame` a partir de la instancia de `RowIterator`

https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe

In [ ]:
# en teoria esto funciona, pero no:
#results_df = results.to_dataframe()

#así que armamos el dataframe a mano:

El `schema` contiene la metadata del resultado, que es la descripción de cada uno de los campos que lo conforman.

In [ ]:
results.schema

In [ ]:
column_headers = [field.name for field in results.schema]
rows = [row.values() for row in results]
results_df = pd.DataFrame(rows, columns=column_headers)

In [ ]:
results_df

Guardamos esos resultados en un archivo local:

In [ ]:
results_df.to_csv('../Data/stackoverflow.csv', sep='\t', index=False)

Vuelvo a ejecutar el job para iterar sobre las filas y mostrar los resultados:

In [ ]:
results = query_job.result()  # Waits for job to complete.

for row in results:
    print("{} : {} views".format(row.url, row.view_count))

## Guardar los resultados de la consulta en un tabla

https://cloud.google.com/bigquery/docs/writing-results?authuser=1#saving_query_results_to_a_table

En la consola de BigQuery https://console.cloud.google.com/bigquery creamos un dataset llamado `resultados_queries` y dentro de ese dataset vamos a escribir los resultados de la consulta en una tabla llamada `prueba_1`

In [ ]:
# cado uno tiene un id distinto acá
project_id = 'clase-53' #'clase-37-286014' 

# TODO(developer): Set table_id to the ID of the destination table.
table_id = project_id + ".resultados_queries.prueba_1"

job_config = bigquery.QueryJobConfig(destination=table_id)

sql = """
    SELECT corpus
    FROM `bigquery-public-data.samples.shakespeare`
    GROUP BY corpus;
"""

# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id))

Podemos descargar los resultados desde la UI 

## Delete table

https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client.delete_table
    


In [ ]:
client.delete_table(table_id, not_found_ok = True)

## Descargar datos de tablas

https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas#download_table_data_using_the_client_library

In [ ]:
import google.auth
from google.cloud import bigquery
#from google.cloud import bigquery_storage_v1beta1

# Explicitly create a credentials object. This allows you to use the same
# credentials for both the BigQuery and BigQuery Storage clients, avoiding
# unnecessary API calls to fetch duplicate authentication tokens.
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Make clients.
bqclient = bigquery.Client(
    credentials=credentials,
    project=your_project_id,
)

#bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient(
#    credentials=credentials
#)

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string(
    "bigquery-public-data.utility_us.country_code_iso"
)
rows = bqclient.list_rows(
    table,
    selected_fields=[
        bigquery.SchemaField("country_name", "STRING"),
        bigquery.SchemaField("fips_code", "STRING"),
    ],
)

# no anda!!
#dataframe = rows.to_dataframe(bqstorage_client=bqstorageclient)

# opción:
column_headers = [field.name for field in rows.schema]
rows_df = [row.values() for row in rows]
dataframe = pd.DataFrame(rows_df, columns=column_headers)

print(dataframe.head())



## Visualización

https://cloud.google.com/bigquery/docs/visualize-jupyter?authuser=1



In [ ]:
query_job = client.query("""
    SELECT
    source_year AS year,
    COUNT(is_male) AS birth_count
    FROM `bigquery-public-data.samples.natality`
    GROUP BY year
    ORDER BY year DESC
    LIMIT 15""")

total_births = query_job.result()  # Waits for job to complete.

column_headers = [field.name for field in total_births.schema]
rows = [row.values() for row in total_births]
total_births_df = pd.DataFrame(rows, columns=column_headers)

In [ ]:
total_births_df.head()

Graficamos el resultado

In [ ]:
%matplotlib inline
total_births_df.plot(kind='bar', x='year', y='birth_count');

In [ ]:
query_job = client.query("""
    SELECT
    wday,
    SUM(CASE WHEN is_male THEN 1 ELSE 0 END) AS male_births,
    SUM(CASE WHEN is_male THEN 0 ELSE 1 END) AS female_births
    FROM `bigquery-public-data.samples.natality`
    WHERE wday IS NOT NULL
    GROUP BY wday
    ORDER BY wday ASC""")

births_by_weekday = query_job.result()  # Waits for job to complete.

column_headers = [field.name for field in births_by_weekday.schema]
rows = [row.values() for row in births_by_weekday]
births_by_weekday_df = pd.DataFrame(rows, columns=column_headers)

In [ ]:
births_by_weekday_df.plot(x='wday');

 Observemos que la cantidad de nacimientos disminuye de forma drástica el domingo (1) y el sábado (7)